## 🎓 Persona : Léa, jeune investisseuse étudiante

**Profil :**
- 👩 24 ans, diplômée de l'EM Lyon
- 💼 Première expérience professionnelle après 2 ans d'alternance
- 💰 Aide parentale pour le financement + épargne personnelle (~15 000 €)
- 🎯 Objectif : réaliser un **premier investissement locatif** dans une **ville étudiante dynamique**

---

### 💡 Objectif d'investissement
> Trouver le **meilleur investissement locatif étudiant** possible avec un **budget global de 200 000 €**,  
> en ciblant un **studio à Lille**, tout en comparant brièvement avec un **T1 à Angers ou Nancy**.

---

### 💰 Hypothèses financières
| Élément | Montant estimé |
|----------|----------------|
| Prix d'achat visé | 160 000 – 180 000 € |
| Apport personnel | 15 000 € |
| Prêt immobilier estimé | 180 000 € sur 20 ans |
| Budget total (frais inclus) | **≈ 200 000 €** |
| Objectif de rentabilité brute | **≥ 5 %** |

---

### 🏙️ Cibles principales
| Ville | Type de bien | Prix moyen au m² | Loyer moyen mensuel | Observations |
|-------|---------------|------------------|---------------------|---------------|
| **Lille** | Studio (20–25 m²) | ~4 500 €/m² | 550–600 € | Marché étudiant tendu, forte demande locative |
| **Angers** | T1 (25–30 m²) | ~3 200 €/m² | 450–500 € | Ville très dynamique, bonne rentabilité brute |
| **Nancy** | T1 (25–30 m²) | ~2 800 €/m² | 420–470 € | Marché abordable, bon rapport prix/rentabilité |

---

### 🔍 Besoins data de Léa
- Identifier **les quartiers les plus rentables** à Lille (ou dans des villes comparables)
- Comparer avec **la rentabilité moyenne en France**
- Analyser l'**évolution du prix au m² et des loyers étudiants** depuis 5 ans
- Calculer la **rentabilité locative brute et nette** par quartier
- Visualiser les **zones à forte concentration étudiante**
- Fournir une **recommandation finale : "où investir avec 200k€ ?"**
- Évaluer le **taux de vacance locative** par quartier pour anticiper les périodes creuses (notamment l'été où les étudiants quittent les logements)
- Analyser la **proximité des transports en commun** et des universités/grandes écoles pour identifier les zones les plus attractives pour les étudiants
- Estimer les **charges de copropriété moyennes** par type de bien et par quartier pour affiner le calcul de rentabilité nette
- Identifier les **opportunités de biens nécessitant des travaux** (décote à l'achat) pour maximiser la plus-value à long terme

---

### 🧭 Objectif du notebook
Créer un outil interactif permettant à Léa de :
1. Comparer la rentabilité d'un **studio à Lille** avec celle d'un **T1 à Angers ou Nancy**  
2. Explorer visuellement les **zones à potentiel locatif élevé**  
3. Obtenir une **recommandation automatique** en fonction de son budget et de ses préférences

## Import des bibliothèques ##

In [1]:
import pandas as pd 
import plotly.express as px
import time
import nbformat

Évaluer le taux de vacance locative en France, et les zones propices à un taux élevé

Constante

In [2]:
BDD = "C:/Users/rivat/OneDrive/Bureau/cour/1 Data Science/Projet"

Lecture fichiers Excel

In [3]:
df_vac = pd.read_excel(f"{BDD}/insee_rp_hist_1968.xlsx", header=1)
df_pop = pd.read_excel(f"{BDD}/POPULATION_MUNICIPALE_COMMUNES_FRANCE.xlsx")
df_communes = pd.read_csv(f"{BDD}/communes-france-2025.csv", sep=",")  # adapte le chemin si nécessaire

C:\Users\rivat\AppData\Local\Temp\ipykernel_19920\2612539266.py:3: DtypeWarning: Columns (1,12,14,16,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_communes = pd.read_csv(f"{BDD}/communes-france-2025.csv", sep=",")  # adapte le chemin si nécessaire


In [4]:
df_communes_coords = df_communes[[
    "code_insee",       # code INSEE pour faire le merge
    "code_postal",      # code postal
    "latitude_centre",  # latitude du centre de la commune
    "longitude_centre"  # longitude du centre de la commune
]]

Mise en forme des colonnes dans les fichiers

In [5]:
df_vac.columns = ["code_commune", "nom_commune", "annee", "part_log_vacant", "col5", "col6", "col7", "col8"]
df_vac = df_vac[["code_commune", "nom_commune", "annee", "part_log_vacant"]]

df_pop.columns = ["objectid", "reg", "dep", "cv", "codgeo",	"libgeo", "p21_pop"]

df_vac = df_vac.dropna(subset=["part_log_vacant"])
df_vac["part_log_vacant"] = pd.to_numeric(df_vac["part_log_vacant"], errors="coerce")
df_vac = df_vac.dropna(subset=["part_log_vacant"])

df_pop["codgeo"] = df_pop["codgeo"].astype(str)
df_vac["code_commune"] = df_vac["code_commune"].astype(str)

Dataframe des villes de plus de 100 000 habitants classées dans l'ordre croissant par rapport au taux de logement vacant.

In [6]:
df_vac = df_vac.sort_values(by="part_log_vacant", ascending=True)

df_pop_100k = df_pop[df_pop["p21_pop"] > 100000]

df_vac_100k = df_vac.merge(
    df_pop_100k[["codgeo", "p21_pop"]],
    left_on="code_commune",
    right_on="codgeo",
    how="inner"
)

df_communes_coords["code_insee"] = df_communes_coords["code_insee"].astype(str)
df_vac_100k["code_commune"] = df_vac_100k["code_commune"].astype(str)

df_vac_100k = df_vac_100k[["code_commune", "nom_commune", "annee", "part_log_vacant", "p21_pop"]]

C:\Users\rivat\AppData\Local\Temp\ipykernel_19920\1603771164.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_communes_coords["code_insee"] = df_communes_coords["code_insee"].astype(str)


Affichage sur la map

In [19]:
# --- Garder uniquement l'année la plus récente pour chaque commune ---
df_vac_100k_recent = df_vac_100k.sort_values("annee", ascending=False) \
                                 .groupby("code_commune", as_index=False) \
                                 .first()  # prend la ligne la plus récente pour chaque commune

# --- Merge avec les coordonnées ---
df_vac_map = df_vac_100k_recent.merge(
    df_communes_coords,
    left_on="code_commune",
    right_on="code_insee",
    how="left"
).dropna(subset=["latitude_centre", "longitude_centre"])

# --- Affichage carte Plotly ---
import plotly.express as px

fig = px.scatter_mapbox(
    df_vac_map,
    lat="latitude_centre",
    lon="longitude_centre",
    size="part_log_vacant",
    color="part_log_vacant",
    hover_name="nom_commune",
    hover_data={
        "p21_pop": True,
        "part_log_vacant": True,
        "code_postal": True,
        "annee": True
    },
    zoom=5,
    height=500,
    color_continuous_scale="OrRd"
)
# --- Top 10 communes avec le plus petit taux de logements vacants ---
top10_low_vac = df_vac_map.sort_values("part_log_vacant", ascending=True).head(10)

# --- Histogramme ---
fig_hist = px.bar(
    top10_low_vac,
    x="nom_commune",
    y="part_log_vacant",
    text="part_log_vacant",
    hover_data={"p21_pop": True, "code_postal": True, "annee": True},
    labels={"part_log_vacant": "Taux logements vacants", "nom_commune": "Commune"},
    title="Top 10 communes avec le plus petit taux de logements vacants",
    color="part_log_vacant",
    color_continuous_scale="Blues",
    height=600
)

fig.update_layout(mapbox_style="open-street-map", margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

fig_hist.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig_hist.show()

C:\Users\rivat\AppData\Local\Temp\ipykernel_19920\332849793.py:17: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/

